# Final Project Check-in 2018-11-16

## Group Name: Lambda

### Student Names
1. Jian Wang
2. Chong Geng
3. Alan Perry
4. Divya Bhargavi
5. Robert Sandor

## Load Data

In [1]:
from math import sqrt
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import operator
import re
from scipy import spatial
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import string
import time

In [2]:
def feature_engineering(train_df, products_df):
    """
    Adds the following features to the training set dataframe: 
    clean_length (the number of words in the 'cleaned' search terms)
    title_length (the number of words in the 'cleaned' title)
    desc_length (the number of words in the 'cleaned' description)
    clean_terms_in_title (the number of time 
    any of the words in clean_terms appears in the title)
    clean_terms_in_desc (the number of time 
    any of the words in clean_terms appears in the description)
    """
    # join the dataframes together
    train_df = train_df.set_index('product_uid').join(products_df.set_index('product_uid'))
    train_df = train_df.reset_index()
    
    # "clean" the search terms of numbers and stop words
    search_terms = train_df['search_term']
    cleaned_terms = [' '.join(tokenize(search_term)) for search_term in search_terms]
    train_df['cleaned_terms'] = cleaned_terms
    
    cleaned = list(train_df['cleaned_terms'])
    title = list(train_df['product_title'])
    desc = list(train_df['product_description'])

    
    # create the features to be used in the model
    train_df['clean_length'] = get_length(cleaned)
    train_df['title_length'] = get_length(title)
    train_df['desc_length'] = get_length(desc)
    train_df['clean_terms_in_title'] = clean_term_in_doc(cleaned, title)
    train_df['clean_terms_in_desc'] = clean_term_in_doc(cleaned, desc)
    
    return train_df

def clean_term_in_doc(terms:list, title:list):
    """
    This cleans the given terms  in the specified document
    """
    count=np.zeros(len(terms))
    for i in range(len(terms)):
        if not pd.isnull(terms[i]): 
            title[i]=title[i].lower()
            for term in terms[i].split(' '):
                if term in title[i].split(' '):
                    count[i]+=1
    return count

def get_length(column):
    """
    This calculates and returns the number of words
    for each row in a specified column
    """
    length = np.zeros(len(column))
    for index in range(len(column)):
        if not pd.isnull(column[index]):
            length[index] = len(column[index].split(' '))
    return length

def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    stops = list(stop_words.ENGLISH_STOP_WORDS)
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words = nopunct.split(" ")
    words = [w for w in words if (len(w) > 2 and (w not in stops))]  # ignore a, an, to, at, be, ...
    return words

def make_dictionary(file):
    '''
    Initiate the glove model as a dictionary
    input: A String which is a file in the project directory
    returns: A dictionary with item = word : 300 d list
    '''
    vecs = dict()
    with open(file) as f:
        lines = f.readlines()
        for word_and_vec in lines:
            elems = word_and_vec.strip().split(' ')
            word = elems[0]
            vec = np.array(elems[1:], dtype=float)
            vecs[word] = vec
    return vecs

In [3]:
products = pd.read_csv('product_descriptions.csv')
train = pd.read_csv('train.csv', encoding='ISO-8859-1')

glove_file = 'glove.6B.300d.txt'
glove_dic = make_dictionary(glove_file)

modified_train = feature_engineering(train, products)

## Fit scikit-learn model

In [4]:
X_train = modified_train[['clean_length', 'title_length', 'desc_length', 'clean_terms_in_title', 'clean_terms_in_desc']]
y_train = modified_train[['relevance']]

In [5]:
# since we can't see the relevancy scores of the test set,
# I decided to split the training set 
train_data, test_data, train_target, test_target = train_test_split(X_train,
                                                                        y_train,
                                                                        random_state=42)

In [6]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(train_data, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
predicted = lin_reg_model.predict(test_data)
print(predicted[:5])
print(test_target[:5])

[[2.12242751]
 [2.35407144]
 [2.41228688]
 [2.67291117]
 [2.66110063]]
       relevance
13534       3.00
29748       2.67
20225       2.67
5169        2.67
49860       2.00


## Evaluation Metric

In [8]:
# since an RMSE function couldn't be found quickly in the sklearn library,
# we just used the MSE function and took the square root of that
rmse_lin_reg = sqrt(mean_squared_error(predicted, test_target))

# this value is equivalent to rank 1756 on the Kaggle leaderboard for this competition
# the benchmark was ~ rank 1680 
rmse_lin_reg

0.5123979817392255